In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from math import sqrt
from model import Model
from rbm import RBM
from rbm_operator import Operator, Sx_, Sy_, Sz_, SzSz_, set_h_Hamiltonian, set_J1_Hamiltonian, set_J2_Hamiltonian

# Simple attempt to implement a variational quantum state calculation 

As an example, we start with a very simple J1/J2 Heisenberg spin chain

We define a class "model" that contains a Hamiltonian and a rule for generating spins according to the MSMS algorithm.

We have something to obtain the wavefunction amplitude $\psi_\theta(s)$ from the spin configuration $s$ that depends on some amplitude $\theta$

From  $\psi_\theta(s)$ for the given spin configuration, we can produce an estimate of the energy of the variational ground state.

Then we can perform a stochastic gradient descent based on it.

Reminder: the operators are defined as 
$$S\cdot S = S_x \cdot S_x + S_y \cdot S_y + S_z \cdot S_z = \frac{1}{2} (S_+ \cdot S_- + S_- \cdot S_+) + S_z \cdot S_z$$

where $S_z = \frac{\hbar}{2} \begin{pmatrix} 1 & 0 \\ 0  & -1 \end{pmatrix}$, $S_+ = \hbar \begin{pmatrix} 0 & 1 \\ 0 & 0 \end{pmatrix} = S_x + S_y$, $S_- = \hbar \begin{pmatrix} 0 & 0 \\ 1 & 0 \end{pmatrix} = S_x - S_y$. A singlet state has energy $-3J/4$ and a triplet state has energy $J/4$

# Implementation of the RBM class

After having a class of spin and Hamiltonian, consider a wavefunction object, take takes a spin and returns a number. The variational quantum state is defined as 

\begin{align*}
\Psi_M(S;W) = \sum_{h_i} e^{\sum_j a_j \sigma^z_j + \sum_i b_i h_i + \sum_{ij} W_{ij} h_i \sigma^{z}_j}
\end{align*}

Here the free parameters of the models are $a_i, b_, W_{ij}$ and $h_1, \dots, h_M$ represents auxilliary spin variables in the network. The internal spins can be explicited traced out to read

\begin{align*}
\Psi(S;W) = e^{\sum_j a_j \sigma^z_j} \times \Pi_{i=1}^M F_i(S)
\end{align*}

where

\begin{align*}
F_i(S) = 2\cosh\left[ b_i + \sum_J W_{ij} \sigma^z_j\right] =  2\cosh\theta_i
\end{align*}

The object would be an NN (most basic example includes the Carleo RBM, which actually has an analytical form) 

I need a function to compute the variational energy. A function to give gradient. And a function to do the gradient descent.


# Operators and derivatives: 

Noting that an operator dcdan be approximated by its local value

\begin{align*}
O_{loc}(s) = \sum_{s, s'}  \left< s\middle| O \middle| s' \right>\frac{\left<s' \middle| \Phi_\theta\right>}{\left<s \middle| \Phi_\theta\right>}
\end{align*}

and an operator's expectation value can be reasonably approximated by 

\begin{align*}
\left<O \right> = \sum_{s} P(s) O_{loc}(s) \approx \frac{1}{M} \sum_{s_i} O_{loc}(s_i)
\end{align*}

Now consider the energy minimization. Let $O_p(s) = \frac{\partial}{\partial \theta_p} \log \left<s\middle|\Psi_\theta\right> = \left<s\middle|O_p \middle| s\right> $,

\begin{align*}
\frac{\partial E(\theta)}{\partial \theta_p} &= 2 \Re \left[ \left\langle E_{\text{loc}}(\mathbf{s}) O^*(\mathbf{s}) \right\rangle - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle \left\langle O^*(\mathbf{s}) \right\rangle \right] \\
&= 2 \Re \left[ \left\langle (E_{\text{loc}}(\mathbf{s}) - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle) O^*(\mathbf{s}) \right\rangle \right]
\end{align*}

To evaluate the derivatives, note that 

\begin{align*}
\frac{\partial}{\partial a_i} \log \left<s\middle|\Psi_\theta\right> &= \sigma_i^z \\
\frac{\partial}{\partial b_j} \log \left<s\middle|\Psi_\theta\right> &= \tanh(\theta_j(S)) \\
\frac{\partial}{\partial W_{ij}} \log \left<s\middle|\Psi_\theta\right> &= \sigma^z_i\tanh(\theta_j(S))
\end{align*}

It follows that the gradient is given by (up to proportionality factors)

\begin{align*}
\frac{\partial E(\theta)}{\partial a_i}  &= \left[ \left\langle E_{\text{loc}}(\mathbf{s}) \sigma_i^z \right\rangle - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle \left\langle \sigma_i^z \right\rangle \right] \\
\frac{\partial E(\theta)}{\partial b_j}  &= \left[ \left\langle E_{\text{loc}}(\mathbf{s}) \tanh(\theta_j(s)) \right\rangle - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle \left\langle \tanh(\theta_j(s)) \right\rangle \right] \\
\frac{\partial E(\theta)}{\partial W_{ij}}  &= \left[ \left\langle E_{\text{loc}}(\mathbf{s}) \sigma^z_i\tanh(\theta_j(s)) \right\rangle - \left\langle E_{\text{loc}}(\mathbf{s}) \right\rangle \left\langle \sigma^z_i\tanh(\theta_j(s)) \right\rangle \right]
\end{align*}


## Sanity check: expectation value of operators 

### Two-site operators (SzSz only, for now)

In [4]:
model = Model(2,2)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(SzSz_(0,0,0,1,model), np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation <SzSz|ud__|SzSz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(SzSz_(0,0,0,1,model), np.array([[[0, 1], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation  <Sz|dd__|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(SzSz_(0,0,0,1,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <SzSz|ud__|SzSz> is (-0.25+0j) with standard deviation 0.0
The average expectation  <Sz|dd__|Sz> is (0.25+0j) with standard deviation 0.0
The average expectation for a 2:1 mixed state is (-0.0125+0j))


### One-site operator (Sz makes sense. Unsure if Sx does, but I'll sweep it under the rug for now.)

In [18]:
model = Model(2,2)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(Sz_(0,0,model), np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation <Sz|u___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(Sz_(0,0,model), np.array([[[0, 1], [1, 0]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation  <Sz|d___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(Sz_(0,0,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sz|u___|Sz> is (0.5+0j) with standard deviation 0.0
The average expectation  <Sz|d___|Sz> is (-0.5+0j) with standard deviation 0.0
The average expectation for a 2:1 mixed state is (0.42+0j))


In [19]:
model = Model(2,2)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(Sz_(1,0,model)+Sz_(0,1,model), np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation <Sz|u___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(Sz_(1,0,model)+Sz_(0,1,model), np.array([[[0, 1], [1, 0]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation  <Sz|d___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(Sz_(1,0,model)+Sz_(0,1,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sz|u___|Sz> is (-1+0j) with standard deviation 0.0
The average expectation  <Sz|d___|Sz> is 0j with standard deviation 0.0
The average expectation for a 2:1 mixed state is (0.335+0j))


In [20]:
operator = Operator(model)
for i in range(2):
    for j in range(2):
        operator += Sz_(i, j, model)
model = Model(2,2)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(operator, np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation <Sz|u___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(operator, np.array([[[0, 1], [1, 0]], [[0, 1], [1, 0]]])) for _ in range(40)]
print(f"The average expectation  <Sz|d___|Sz> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(operator, batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sz|u___|Sz> is 0j with standard deviation 0.0
The average expectation  <Sz|d___|Sz> is 0j with standard deviation 0.0
The average expectation for a 2:1 mixed state is (0.88+0j))


In [21]:
#NB: This assumes that the spin is up for probability 2/3 and down for probabilty 1/3
#If you set them to be the same the spin expectation is 1/2, which checks out for |up>+|dn>
model = Model(2,3)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(Sx_(1,0,model), np.array([[[1, 0], [0, 1], [1, 0]], [[0, 1], [1, 0], [1,0]]])) for _ in range(40)]
print(f"The average expectation <Sx|u___|Sx> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(Sx_(1,0,model), np.array([[[1, 0], [0, 1], [1, 0]], [[0, 1], [1, 0], [1,0]]])) for _ in range(40)]
print(f"The average expectation  <Sx|d___|Sx> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(100) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(Sx_(0,0,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sx|u___|Sx> is (0.5+0j) with standard deviation 0.0
The average expectation  <Sx|d___|Sx> is (0.5+0j) with standard deviation 0.0
The average expectation for a 2:1 mixed state is (0.3575000000000001+0j))


In [24]:
model = Model(2,3)
rbm = RBM(model)
average_expectations = [rbm.expectation_value(Sy_(1,0,model), np.array([[[1, 0], [0, 1], [1, 0]], [[0, 1], [1, 0], [1,0]]])) for _ in range(40)]
print(f"The average expectation <Sy|u___|Sy> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
average_expectations = [rbm.expectation_value(Sy_(1,0,model), np.array([[[1, 0], [0, 1], [1, 0]], [[0, 1], [1, 0], [1,0]]])) for _ in range(40)]
print(f"The average expectation  <Sy|d___|Sy> is {np.mean(average_expectations)} with standard deviation {np.std(average_expectations)}")
rbm = RBM(model)
batch = rbm.create_batch(200) #This uses evaluate_dummy, which gives 2/3 for spin up and 1/3 for spin down 
average_expectations = [rbm.expectation_value(Sy_(1,0,model), batch[i]) for i in range(len(batch))]
print(f"The average expectation for a 2:1 mixed state is {np.mean(average_expectations)})")

The average expectation <Sy|u___|Sy> is -0.5j with standard deviation 0.0
The average expectation  <Sy|d___|Sy> is -0.5j with standard deviation 0.0
The average expectation for a 2:1 mixed state is 0.01j)


# Implementation of the learning process

Idea:

For each iteration:

Start with a set of weights.

For each weight, initialize the batch of state configurations based on the MCMC. For each state, get the amplitude. The combination gives an estimation of energy. This energy generates an estimation of gradient descent. Change the weight. 



In [71]:
model = Model(2,3)
rbm = RBM(model)

N = 100

#create a batch
batch = rbm.create_batch(N)
# Implement a Hamiltonian

Ham =  set_J1_Hamiltonian(model, J = 1) + set_h_Hamiltonian(model, h = 4)

Szs = set_h_Hamiltonian(model, h = 1)

print(f"the spin expectation value is {rbm.expectation_value_batch(Szs, batch)}")

def calculate_Sz_expectation_brute_force(spins):
    return  np.mean(np.sum(batch[:, :, :, 0]/2 - batch[:, :, :, 1]/2, axis=(1, 2)))

print(f"the naive spin expectation value is {calculate_Sz_expectation_brute_force(batch)}")
      

the spin expectation value is (0.04+0j)
the naive spin expectation value is 0.04


In [39]:
test = [np.mean(np.sum(rbm.create_batch(400, burn_in = 100, skip = 10)[:,:,:,0], axis = (1,2))) for _ in range(40)]
print(np.mean(test), np.std(test))

3.4515625 0.05705449012785935


### Symmetry of the RBM.

Consider a translation operator working as $\sigma_j(k) = T_k \sigma_j$. An obvious requirement for translation symmetry is that $\Psi_\theta(\sigma) = \Psi_\theta(T_s\sigma)$, and an obvious way to implement this is to just artificially sum over contributions on the output $\sum_i \Psi_\theta(T_{s_i}\sigma)$, but this won't improve the efficiency of the algorithm.

A more tractable wa is to use convolutions. Before we have RBM as 

\begin{align*}
\Psi_M(S;W) &= \sum_{h_i} e^{\sum_j a_j \sigma^z_j + \sum_i b_i h_i + \sum_{ij} W_{ij} h_i \sigma^{z}_j}\\
\end{align*}

Now, we can rewrite it as below, where $f = 1, \alpha_s$ is a number of feature maps, and in particular $W_{j}^{(f)} $ has $\alpha_s \times N$ elements. Note that because the spins are all summed over all translations this is translation invariant.

\begin{equation}
\Psi_{\alpha}(\mathbf{S}; \mathbf{W}) = \sum_{h_{i,s}} \exp \left[ \sum_{f}^{a} \left( a^{(s)} \sum_{s}^{S} \sum_{j}^{N} \tilde{\sigma}_{j}^{z}(s) + b_{f}^{(s)} h_{f,s} + \sum_{s}^{S} \sum_{j}^{N} h_{f,s} W_{j}^{(f)} \tilde{\sigma}_{j}^{z}(s) \right) \right],
\end{equation}
